## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-20-56-24 +0000,nyt,Majority of Latin Americans Endorsed Trump’s I...,https://www.nytimes.com/2026/01/21/world/ameri...
1,2026-01-21-20-55-33 +0000,bbc,Seven more countries agree to join Trump's Boa...,https://www.bbc.com/news/articles/cn8jek4vv8ko...
2,2026-01-21-20-51-42 +0000,nypost,House committee finds Bill and Hillary Clinton...,https://nypost.com/2026/01/21/us-news/house-co...
3,2026-01-21-20-50-11 +0000,nyt,House Panel Votes to Hold Clintons in Contempt...,https://www.nytimes.com/2026/01/21/us/politics...
4,2026-01-21-20-47-41 +0000,nyt,Live Updates: Trump Drops Tariff Threat Over G...,https://www.nytimes.com/live/2026/01/21/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2363/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,104
36,greenland,42
44,davos,28
53,court,20
283,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
208,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...,221
6,2026-01-21-20-46-06 +0000,cbc,"Trump reiterates desire to acquire Greenland, ...",https://www.cbc.ca/news/world/trump-davos-spee...,214
81,2026-01-21-18-53-24 +0000,nypost,"Bessent calls Denmark ‘irrelevant,’ shrugs off...",https://nypost.com/2026/01/21/business/bessent...,201
92,2026-01-21-18-34-00 +0000,wsj,‘A Piece of Ice for World Protection’: Trump D...,https://www.wsj.com/politics/policy/a-piece-of...,200
136,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...,199


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
208,221,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...
75,84,2026-01-21-19-00-40 +0000,latimes,Supreme Court wary of Trump's bid to fire Fed ...,https://www.latimes.com/politics/story/2026-01...
11,69,2026-01-21-20-39-00 +0000,wsj,A House committee voted to find former Preside...,https://www.wsj.com/politics/republicans-turn-...
278,63,2026-01-21-08-38-29 +0000,wsj,Stock Market Today: Dow Jumps After Trump Call...,https://www.wsj.com/livecoverage/stock-market-...
323,59,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
136,51,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...
261,47,2026-01-21-09-55-53 +0000,bbc,Train driver killed in second deadly Spain rai...,https://www.bbc.com/news/articles/c1m78xl0gmpo...
80,44,2026-01-21-18-54-48 +0000,nypost,Barron Trump ‘saved’ friend’s life by calling ...,https://nypost.com/2026/01/21/us-news/barron-t...
248,40,2026-01-21-10-30-00 +0000,wsj,Google searches for “No Buy January” have hit ...,https://www.wsj.com/economy/consumers/no-buy-j...
19,39,2026-01-21-20-26-07 +0000,nypost,Trial of NJ tech boss accused of murdering bro...,https://nypost.com/2026/01/21/us-news/trial-of...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
